<center>
    <h1 style="color:deeppink">Just between us SQL Friends...</h1>
    <h2 style="color:deeppink">RuPaul's Drag Race: Visualized</h2>
    <img src="media/header.gif" height="100px">
</center>

<h2 style="color:deeppink">Import Libraries & Connect SQLite Database</h2>

In [411]:
import pandas as pd
import sqlite3
from pandas.io.json import json_normalize

drag_race_data_base = sqlite3.connect("drag_race_data.db")

<h2 style="color:deeppink">Define Constants, Global Variables, and API endpoints</h2>

In [450]:
DRAG_RACE_API_URL = "http://www.nokeynoshade.party/api/"

#API Endpoints
SEASONS = "seasons"
QUEENS = "queens/all"
SEASON_BY_NUMBER = "seasons/{}"

<h2 style="color:deeppink">Define Functions</h2>

In [451]:
def get_data(endpoint, SEASON = None):
    data = pd.read_json(DRAG_RACE_API_URL + endpoint.format(SEASON))
    return data


def get_seasons():
    df = pd.DataFrame()
    temp_df = pd.DataFrame()
    for i in range(len(season_id_df)):
        temp_df = get_data("seasons/{}", i + 1)
        temp_df = json_normalize(temp_df['queens'])
        temp_df['season_id'] = i + 1
        df = pd.concat([df, temp_df], sort=False)
        i += 1
    return df


def csv_export(df, file_name):
    df.to_csv(file_name, index=True)

<h2 style="color:deeppink">Define SQL Queries</h2>

In [465]:
all_queens_query = """
    SELECT queens.id, name, season_number, place, winner, missCongeniality AS miss_congeniality, image_url, quote FROM seasons
    INNER JOIN season_ids ON seasons.season_id = season_ids.season_id
    INNER JOIN queens ON seasons.id = queens.id
    ORDER BY season_number, place DESC
"""


all_stars_query = """
    SELECT queens.id, name, season_number, place, winner, missCongeniality AS miss_congeniality, image_url, quote FROM seasons
    INNER JOIN season_ids ON seasons.season_id = season_ids.season_id
    INNER JOIN queens ON seasons.id = queens.id
    WHERE season_number LIKE "%A%"
    ORDER BY season_number, place DESC
"""


main_seasons_query = """
    SELECT queens.id, name, season_number, place, winner, missCongeniality AS miss_congeniality, image_url, quote FROM seasons
    INNER JOIN season_ids ON seasons.season_id = season_ids.season_id
    INNER JOIN queens ON seasons.id = queens.id
    WHERE season_number IN (1, 11)
    ORDER BY season_number, place DESC
"""


main_season_winners_query = """
    SELECT queens.id, name, season_number, place, winner, missCongeniality AS miss_congeniality, image_url, quote FROM seasons
    INNER JOIN season_ids ON seasons.season_id = season_ids.season_id
    INNER JOIN queens ON seasons.id = queens.id
    WHERE winner IS True AND season_number NOT LIKE "A%" AND place = 1
"""


all_stars_winners_query = """
    SELECT queens.id, name, season_number, place, winner, missCongeniality AS miss_congeniality, image_url, quote FROM seasons
    INNER JOIN season_ids ON seasons.season_id = season_ids.season_id
    INNER JOIN queens ON seasons.id = queens.id
    WHERE season_number LIKE "A%" AND place = 1
"""


miss_congeniality_query = """
    SELECT * FROM seasons
    INNER JOIN season_ids ON seasons.season_id = season_ids.season_id
""" + """
    INNER JOIN queens ON seasons.id = queens.id
    WHERE missCongeniality IS True
    ORDER BY season_id"""

<h2 style="color:deeppink">Define values used for cleanup</h2>

In [453]:
queens_drop_cols = ['queens', 'name']

<h2 style="color:deeppink">Create raw dataframes from API endpoints</h2>

In [458]:
raw_seasons_df = get_data(SEASONS)
raw_queens_df = get_data(QUEENS)

# create season_id_df to look up season numbers by id
season_id_df = pd.DataFrame({"season_id": raw_seasons_df['id'], "season_number": raw_seasons_df['seasonNumber']})

<h2 style="color:deeppink">Clean and organize the data</h2>

<h2 style="color:deeppink">Create SQL Tables</h2>

In [459]:
# queens table
raw_queens_df.to_sql('queens', drag_race_data_base, if_exists='replace')

# seasons table
seasons_df = get_seasons().set_index('id').drop(queens_drop_cols, axis=1)
seasons_df.to_sql('seasons', drag_race_data_base, if_exists = 'replace')

# season_ids table
season_id_df.to_sql('season_ids', drag_race_data_base, if_exists='replace')

<h2 style="color:deeppink">Create DataFrames from the SQL Tables</h2>

In [460]:
#all queens, ranked
main_seasons_df = pd.read_sql(main_seasons_query, drag_race_data_base)
all_stars_df = pd.read_sql(all_stars_query, drag_race_data_base)

#winners
main_season_winners_df = pd.read_sql(main_season_winners_query, drag_race_data_base).astype({'season_number':'int32'}).sort_values(by=['season_number'])
all_stars_winners_df = pd.read_sql(all_stars_winners_query, drag_race_data_base).sort_values(by=['season_number'])

<h2 style="color:deeppink">Sandbox</h2>
<b style="color:deeppink">Use the space below to experiment. Nothing below will be included in the finished product.</b>

In [468]:
miss_congeniality_df = pd.read_sql(miss_congeniality_query, drag_race_data_base)
miss_congeniality_df

,id,place,season_id,index,season_id,season_number,index,id,name,winner,missCongeniality,image_url,quote
0,11.0,2.0,1,0,1,1,7,11,Nina Flowers,0,1,http://www.nokeynoshade.party/images/nina-flow...,I don't consider myself as a female impersonat...
1,20.0,5.0,2,3,2,2,16,20,Pandora Boxx,0,1,http://www.nokeynoshade.party/images/pandora-b...,I'm Pandora Boxx and I'm thirty-bleep years old.
2,33.0,4.0,3,4,3,3,29,33,Yara Sofia,0,1,http://www.nokeynoshade.party/images/yara-sofi...,""""""
3,46.0,4.0,4,6,4,4,42,46,Latrice Royale,0,1,http://www.nokeynoshade.party/images/latrice-r...,Jesus is a biscuit! Let him sop you up!
4,20.0,11.0,5,1,5,A1,16,20,Pandora Boxx,0,1,http://www.nokeynoshade.party/images/pandora-b...,I'm Pandora Boxx and I'm thirty-bleep years old.
5,33.0,5.0,5,1,5,A1,29,33,Yara Sofia,0,1,http://www.nokeynoshade.party/images/yara-sofi...,""""""
6,46.0,7.0,5,1,5,A1,42,46,Latrice Royale,0,1,http://www.nokeynoshade.party/images/latrice-r...,Jesus is a biscuit! Let him sop you up!
7,57.0,7.0,6,7,6,5,53,57,Ivy Winters,0,1,http://www.nokeynoshade.party/images/ivy-winte...,Hey ladies! Oh my god!
8,75.0,5.0,7,8,7,6,68,75,BenDeLaCreme,0,1,http://www.nokeynoshade.party/images/benda-la-...,"Excuse me, we originated the language"
9,90.0,5.0,8,9,8,7,83,90,Katya Zamolodchikova,0,1,http://www.nokeynoshade.party/images/katya.jpg,I’m trying to just serve sexy and confident wh...


<h2 style="color:deeppink">Export to CSV</h2>

In [448]:
# csv_export(DataFrame, file_name)